In [1]:
!pip install transformers torch sentence-transformers faiss-cpu wikipedia datetime pytz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 932.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 56.7 MB/s eta 0:00:00
  

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import wikipedia
from datetime import datetime
import pytz
import random
import re
import warnings
warnings.filterwarnings('ignore')

class TerraMind:
    def __init__(self):
        # Model Initialization (Optimized for speed)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model_name = "microsoft/DialoGPT-large"  # More powerful than medium
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.model = AutoModelForCausalLM.from_pretrained(self.model_name).to(self.device)
        self.generator = pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            device=0 if self.device == "cuda" else -1,
        )

        # Knowledge Retrieval (RAG - Retrieval Augmented Generation)
        self.retriever = SentenceTransformer('all-MiniLM-L6-v2').to(self.device)
        self.knowledge_index = faiss.IndexFlatL2(384)  # Vector dimension for MiniLM

        # Conversation Memory
        self.chat_history = []
        self.max_history = 6  # Last 3 exchanges

        # Bot Personality & Config
        self.bot_name = "TerraMind"
        self.user_name = "Friend"  # Warm, nature-themed username
        self.timezone = pytz.timezone('UTC')  # Can be changed

        # Predefined Responses & Skills
        self.skills = {
            "time": self.get_current_time,
            "date": self.get_current_date,
            "weather": self.get_weather_response,
            "joke": self.tell_joke,
            "calculate": self.simple_math,
            "wiki": self.get_wikipedia_summary,
            "fact": self.tell_nature_fact,
        }

        # Welcome Message
        self.welcome_msg = f"""
        🌿 **Welcome to {self.bot_name} - Where Knowledge Grows Naturally** 🌱
        - Ask me anything (nature, science, jokes, time, etc.)
        - Type `/help` for commands
        - Type `quit` to return to the forest
        Current Season: {self.get_season()} | Forest Time: {self.get_current_time()}
        """

    # ==== Nature-Themed Utility Functions ====
    def get_current_time(self):
        return datetime.now(self.timezone).strftime("%H:%M:%S")

    def get_current_date(self):
        return datetime.now(self.timezone).strftime("%Y-%m-%d")

    def get_season(self):
        month = datetime.now().month
        if 3 <= month <= 5:
            return "Spring 🌸"
        elif 6 <= month <= 8:
            return "Summer ☀️"
        elif 9 <= month <= 11:
            return "Autumn 🍂"
        else:
            return "Winter ❄️"

    def get_weather_response(self):
        responses = [
            "I don’t have real-time weather, but the trees whisper it's a gentle day.",
            "The wind carries no storms today. It's safe to wander.",
            "Nature's mood is calm. A perfect time to explore!"
        ]
        return random.choice(responses)

    def tell_joke(self):
        jokes = [
            "Why did the leaf go to the doctor? It was feeling green!",
            "What did the tree wear to the pool party? Swimming trunks!",
            "Why don't trees use social media? They prefer face-to-bark communication."
        ]
        return random.choice(jokes)

    def tell_nature_fact(self):
        facts = [
            "A single oak tree can produce 10 million acorns in its lifetime.",
            "The Amazon Rainforest produces 20% of the world's oxygen.",
            "Bamboo can grow up to 35 inches in a single day!"
        ]
        return f"🌍 Nature's Secret: {random.choice(facts)}"

    def simple_math(self, query):
        try:
            expr = re.sub(r'[^0-9+\-*/(). ]', '', query)
            return f"🌱 Growth Calculation: {eval(expr)}"  # Caution: Basic eval usage
        except:
            return "🍃 Couldn't grow that number. Try something like '5 + 3'."

    def get_wikipedia_summary(self, query):
        try:
            summary = wikipedia.summary(query, sentences=2)
            return f"📚 Forest Archives: {summary}"
        except:
            return "🌳 No roots of knowledge found. Try another query."

    # ==== Response Generation Logic ====
    def retrieve_knowledge(self, query):
        """Retrieve relevant facts if available"""
        if "time" in query.lower():
            return f"⏳ Forest Time: {self.get_current_time()}"
        elif "date" in query.lower():
            return f"📅 Natural Cycle: {self.get_current_date()}"
        elif "weather" in query.lower():
            return self.get_weather_response()
        elif "joke" in query.lower():
            return self.tell_joke()
        elif "fact" in query.lower():
            return self.tell_nature_fact()
        elif any(op in query for op in ['+', '-', '*', '/']):
            return self.simple_math(query)
        elif "who is" in query.lower() or "what is" in query.lower():
            return self.get_wikipedia_summary(query.split("is")[-1].strip())
        return None

    def generate_response(self, user_input):
        """Generate AI response with context"""
        # Check for commands
        if user_input.lower() in ['quit', 'exit', 'bye']:
            return "🌲 Farewell, Friend. May your path be evergreen."

        # Check for help command
        if user_input.lower() == '/help':
            return (
                "🪴 **TerraMind's Root Commands:**\n"
                "- Ask me anything (nature, science, history, etc.)\n"
                "- `time` → Current forest time\n"
                "- `date` → Today's natural cycle\n"
                "- `weather` → Nature's mood\n"
                "- `joke` → Earthy humor\n"
                "- `fact` → Fascinating nature facts\n"
                "- `calculate <expression>` → Math solver\n"
                "- `who is <person>` → Forest archives lookup\n"
            )

        # Retrieve knowledge if applicable
        knowledge_response = self.retrieve_knowledge(user_input)
        if knowledge_response:
            return knowledge_response

        # Generate AI response with conversation history
        prompt = (
            f"{' '.join(self.chat_history[-self.max_history:])}\n"
            f"Friend: {user_input}\n"
            f"TerraMind:"
        )

        response = self.generator(
            prompt,
            max_length=150,
            num_return_sequences=1,
            temperature=0.8,
            top_k=50,
            top_p=0.9,
            repetition_penalty=1.2,
            do_sample=True,
        )

        # Extract and clean response
        ai_response = response[0]['generated_text'].split("TerraMind:")[-1].strip()
        ai_response = re.sub(r'<\|.*?\|>', '', ai_response)  # Remove special tokens

        # Ensure response isn't just repeating input
        if ai_response.lower() == user_input.lower():
            return "🌱 The soil of your question is rich. Tell me more."

        return ai_response

    # ==== Main Chat Loop ====
    def start_chat(self):
        print(self.welcome_msg)

        while True:
            try:
                user_input = input(f"{self.user_name}: ").strip()
                if not user_input:
                    continue

                # Get response
                response = self.generate_response(user_input)
                print(f"{self.bot_name}: {response}")

                # Update chat history
                self.chat_history.extend([user_input, response])
                if len(self.chat_history) > self.max_history * 2:
                    self.chat_history = self.chat_history[-self.max_history * 2:]

                # Exit condition
                if user_input.lower() in ['quit', 'exit', 'bye']:
                    break

            except KeyboardInterrupt:
                print(f"\n{self.bot_name}: 🍃 The wind carries you away. Return anytime.")
                break
            except Exception as e:
                print(f"{self.bot_name}: 🪻 A small storm disrupted us. Let's regrow our conversation.")
                continue

In [ ]:
# Initialize & run
nature_bot = TerraMind()
nature_bot.start_chat()

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]


        🌿 **Welcome to TerraMind - Where Knowledge Grows Naturally** 🌱
        - Ask me anything (nature, science, jokes, time, etc.)
        - Type `/help` for commands
        - Type `quit` to return to the forest
        Current Season: Spring 🌸 | Forest Time: 07:04:53
        
Friend: who is joe bidden?
TerraMind: 📚 Forest Archives: Joseph Robinette Biden Jr. (born November 20, 1942) is an American politician who was the 46th president of the United States from 2021 to 2025.
Friend: what date is it?
TerraMind: 📅 Natural Cycle: 2025-05-12
